# Actual Causality: the modified Halpern-Pearl definition

**Summary**

Here we show how the tools made available within Causal Pyro  TODO: CHANGE NAME(?) can be used to implement the notion of actual causality developed by Halpern and Pearl (see J. Halpern, *Actual Causality*, 2016), and illustrate its workings by replicating a few key examples from the book.

**Outline**

[Intuitions](##intuitions)
    
[Formalization](#formalization)

- [Structural causal models](#structural-causal-models)

- [Halpern-Pearl modified definition of actual causality](#halpern-pearl-modified-definition-of-actual-causality)

[Implementation](#implementation)

[Examples](#examples)

- [Comments on example selection](#comments-on-example-selection)
  
- [Stone-throwing](#stone-throwing)

- [Forest fire](#forest-fire)

- [Doctors](#doctors)

- [Friendly fire](#friendly-fire)





## Intuitions

Actual causality (sometimes called **token causality** or **specific causality**) is usually contrasted with type causality (sometimes called **general causality**). While the latter is concerned with general statements (such as "smoking causes cancer"), actual causality focuses on particular events. For illustration, consider the following causality-related questions:

- **Friendly Fire**: On March 24, 2002, A B-52 bomber fired a Joint Direct Attack Munition at a US battalion command post, killing three and injuring twenty special forces soldiers. Out of multiple potential contributing factors, which were **actually** responsible for the incident?
  
- **Schizophrenia** : The disease arises from the interaction between multiple genetic and environmental factors. Given a particular patient and what we know about them, which of these factors **actually** caused her state?
  
- **Explainable AI**: Your loan application has been refused. The bank representative informs you the decision was made using predictive modeling to estimate the probability of default. They give you a list of various factors considered in the prediction. But which of these factors **actually** resulted in the rejection, and what were their contributions?
  
These are questions about **actual causality**. While having answers to such questions is not directly useful for prediction tasks, they are useful for understanding how we can prevent undesirable outcomes similar to ones that we have observed or promote the occurrence of desirable outcomes in contexts similar to the ones in which they had been observed. These context-sensitive causality questions are also an essential element of blame and responsibility assignments, and of at least one prominent account of the notion of explanation (all of which will be explored in other notebooks). TODO add links

The general intuition behind the notion of actual causality that we will focus on is that a certain state of antecedent nodes is the cause of a given state of the consequent nodes if there is a part of the actual reality such that if it is kept fixed at what it actually is, and we intervened on the antecedent nodes to be in a different state, the consequent nodes would no longer be in the observed states. A proper explication of this notion requires the context of structural causal models -  we first explain what these are, and then move on to the definition.

## Formalization 

### Structural causal models

While statistical information might help address questions of actual causality, is not sufficient.  One requires causal theories that explain how the relevant aspects of the world function, as well as information about the actual facts pertaining to the specific case. For this reason, the notion on which we focus in this notebook is formulated within the framework of structural causal models, which can represent such information.

The notion is defined in the context of a deterministic structural causal model (SCMs). One major component thereof is a selection of **variables**. For instance, in a very simple model for a forest-fire problem, we might consider a model with three endogenous binary variables: $FF$ (forest fire), $L$ (lightning), and $MD$ (match dropped) whose values are determined by the values of other variables, and two exogenous noise variables $U_{MD}$ and $U_L$ that determine the values of $MD$ and $L$. Moreover, some of those variables/nodes are connected by means of directed **edges**. For instance, in the example at hand, the model contains two edges that go from $U_MD$ to $MD$ and from $U_L$ to $L$ respectively, and two edges that go from $L$ to $FF$ and from $MD$ to $FF$. Each influence is associated with a **structural equation** - for instance, $FF = max(L, MD)$ indicates that a forest fire occurs if either of the two factors occurs. SCMs come also with a **context**, which is the values of **exogenous variables** whose values are not determined by the structural equations, but rather by factors outside the model. In our example, one context might be that both a match has been dropped and a lightning occurred.

More formally, a causal model $M$ is a tuple $\langle S, F\rangle$, where:

- $S$ is a **signature**, that is a tuple $\langle U, V, R\rangle$, where $U$ is a set of exogenous variables, $V$ is a set of endogenous variables and $R: U \cup V \mapsto R(Y)$, where $R(Y)\neq \emptyset$, that is $R$ assigns non-empty ranges to exogenous and endogenous variables.

- To each endogenous $X\in V$, $F$ assigns a function $F_X$, which maps the cross-product of ranges of all variables other than $X$ to $R(X)$. In other words, $F_X$ determines the value of $X$ given the values of other variables in the model (some of them might be redundant in a given equation). The intuition is that these functions correspond to structural equations of the form $X = F_X(U, V)$ which are to be read from right to left: if the values of $U\cup V$ are fixed to be such-and-such, say $\vec{u}$ and $\vec{v}$, this causes $X$ to take the value $F_X(\vec{u}, \vec{v})$.

A **deterministic causal model** (also called **causal setting**), $\langle M, \vec{u}\rangle$ is a causal model $M$ together with fixed settings $\vec{u}$ of its exogenous variables $U$. To intervene, say, to make $Y$ have value $y$, is to replace the structural equation for $Y$ of the form $Y = F_Y(U, V)$ with $Y = y$. $\langle M, \vec{u}\rangle \models [Y \leftarrow y](X = x)$ means: in the deterministic model obtained from $\langle M, \vec{u}\rangle$ by intervening on $Y$ to have value $y$ $X$ has value $x$. Sometimes, instead of $X = x$, one might be interested in a more general claim $\varphi$ involving potentially multiple variables, in which case the notation is $\langle M, \vec{u}\rangle \models [Y \leftarrow y](\varphi)$. 

## Halpern-Pearl modified definition of actual causality

It is important to recognize that the straightforward counterfactual strategy, which asks whether the event would have occurred if the antecedent had not taken place, is inadequate as a definition of actual causality. A simple example can help illustrate this point. Suppose I throw a stone, which hits and shatters a bottle. However, just a second later, Bill also throws a stone at the bottle but misses, solely because the bottle was already shattered by my stone. In this scenario, the intuition is that my throw is the cause of the bottle shattering, even though the bottle would still have shattered if I hadn't thrown the stone. 
This highlights the need for a more elaborate account that considers the actual state, taking into consideration the fact that Bill's stone did not, in fact, hit the bottle. One such account involves the following definition of actual causality:

Given an SCM $M$ and a vector of its exogenous variable settings $\vec{u}$ we'll write $(M, \vec{u})\models [ \vec{Y} \leftarrow \vec{y}]\psi$ just in case $\psi$ holds in $(M',\vec{u})$, where $M'$ is the intervened model obtained by replacing the structural equation(s) for $\vec{Y}$ in $M$ with $\vec{Y_i} = \vec{y_i}$. 

We say that $\vec{X}=\vec{x}$ is an actual cause of $\varphi$ in $(M,\vec{u})$ just in case:

AC1. Factivity: $(M, \vec{u}) \models [\vec{X} = \vec{x} \wedge \varphi]$

AC2. Necessity:

$\exists \vec{W}, \vec{x}'(M, \vec{u})\models [\vec{X} \leftarrow \vec{x}', \vec{W} = \vec{w}^{\star}]   \neg \varphi$,
where $\vec{w}^\star$ are the actual values of $\vec{W}$, i.e. $(M, \vec{u}) \models \vec{W} = \vec{w}^\star$

AC3. Minimality: $\vec{X}$ is a subset-minimal set of potential causes satisfying AC2

AC1 requires that both the antecedent and the consequent hold. The intuition behind AC2 is that for $\vec{X}=\vec{x}$ to be the actual cause of $\varphi$, there needs to be a vector of witness nodes $\vec{W}$ and a vector $\vec{x'}$ of *alternative* settings of $\vec{X}$ such that if $\vec{W}$ are intervened to have their actual values $\vec{w^\star}$, and $\vec{X}$ are intervened to have values $\vec{x'}$, $\varphi$ no longer holds in the resulting model. AC3 requires that the antecedent should be a minimal one satisfying AC2.

## Implementation


In [1]:
import functools

import numpy as np
from itertools import combinations


import torch
from typing import Dict, List, Optional,  Union, Callable

import pandas as pd

import pyro
import pyro.distributions as dist
from chirho.indexed.ops import IndexSet, gather, indices_of, scatter
from chirho.interventional.handlers import do
from chirho.counterfactual.handlers.counterfactual import MultiWorldCounterfactual, Preemptions, BiasedPreemptions
from chirho.observational.handlers import condition

Here and in later notebooks, instead of full enumeration, we will be approximating the answers with sampling. In particular, answering an actual causality query requires investigating the consequences of intervening on all possible witness candidate nodes in all possible combinations to have the values they actually have in a given model. While complete enumeration would work for smaller models, we implement a more general approximate method, which draws random sets of witness nodes multiple times. For smaller models (as the one used in our examples), complete coverage of all possible combinations is easily obtained. For larger models complete enumeration becomes less feasible.

An SCM in this context is represented by a Pyro model, where the exogenous variables are stochastic and introduced using `pyro.sample`, and all the endogenous variables are determined by these, and introduced by `pyro.deterministic` (read on for examples). For simplicity we also assume the antecedent nodes are binary (this assumption can be weakened to them being discrete), and that the consequent nodes are discrete. 

The key role in this implementation is played by the preemption handler, which is used to randomly select some of the witness candidate nodes and preempt them - in our case, with their observed values.

The key moves are in our `def __call__`:

- `pyro.plate` is a messenger used to construct conditionally independent sequences of variables, we use it to obtain multiple independent samples.

- `pyro.condition` is used to constrain the values of some variables. We use it to fix the values of the exogenous variables. As they are causally upstream from all the other variables, this also fixes the values of the endogenous variables.
  
- `Preemptions` is a messenger that in post-processing fixes a random selection of (witness) nodes to the results of certain actions - in our case, these actions correspond to the actual values of the variables.
  
- `do`  is a handler that intervenes on selected variables to have the specified values. We use it to fix the *alternative* combination of values for the antecedent nodes.
  
- `MutliWorldCounterfactual` allows us to keep track of all the scenarios, keeping split records thereof as values at various sites.

- `pyro.poutine.trace` is a messenger that keeps track of sites visited for downstream use.

- within the model, we compare the observed and the intervened consequent and record whether these differ.
  


In [2]:
class HalpernPearlModifiedApproximate:

    def __init__(
        self, 
        model: Callable,
        antecedents: Union[Dict[str, torch.Tensor], List[str]],
        outcome: str,
        witness_candidates: List[str],
        observations: Optional[Dict[str, torch.Tensor]],
        sample_size: int = 100,
        event_dim: int = 0
        ):
        
        self.model = model
        self.antecedents = antecedents
        self.outcome = outcome
        self.witness_candidates = witness_candidates
        self.observations = observations
        self.sample_size = sample_size

        self.antecedents_dict = (
            self.antecedents if isinstance(self.antecedents, dict)
            else self.revert_antecedents(self.antecedents)
        )
    
        self.preemptions = {candidate: functools.partial(self.preempt_with_factual,
                                             antecedents = self.antecedents) for 
                                             candidate in self.witness_candidates}
        

    @staticmethod
    def revert_antecedents(antecedents: List[str]) -> Dict[str, Callable[[torch.Tensor], torch.Tensor]]:
        return {antecedent: (lambda v: 1 - v) for antecedent in antecedents}

    @staticmethod   
    def preempt_with_factual(value: torch.Tensor, *,
                          antecedents: List[str] = None, event_dim: int = 0):
    
        if antecedents is None:
            antecedents = []

        antecedents = [a for a in antecedents if a in indices_of(value, event_dim=event_dim)]

        factual_value = gather(value, IndexSet(**{antecedent: {0} for antecedent in antecedents}),
                                event_dim=event_dim)
            
        return scatter({
            IndexSet(**{antecedent: {0} for antecedent in antecedents}): factual_value,
            IndexSet(**{antecedent: {1} for antecedent in antecedents}): factual_value,
        }, event_dim=event_dim)
        
        
    def __call__(self, *args, **kwargs):
        with pyro.poutine.trace() as trace:
            with MultiWorldCounterfactual():
                with do(actions=self.antecedents_dict):
                    with Preemptions(actions = self.preemptions):
                        with pyro.condition(data={k: torch.as_tensor(v) for k, v in self.observations.items()}):
                            with pyro.plate("plate", self.sample_size):
                                self.consequent = self.model()[self.outcome]
                                self.intervened_consequent = gather(self.consequent, IndexSet(**{ant: {1} for ant in self.antecedents}))
                                self.observed_consequent = gather(self.consequent, IndexSet(**{ant: {0} for ant in self.antecedents}))
                                self.consequent_differs = self.intervened_consequent != self.observed_consequent   
                                pyro.factor("consequent_differs", torch.where(self.consequent_differs, torch.tensor(0.0), torch.tensor(-1e8)))
                            
        self.trace = trace.trace

        # slightly hacky solution for odd witness candidate sets
        if  isinstance(self.consequent_differs.squeeze().tolist(), bool):
            self.existential_but_for = self.consequent_differs.squeeze()
        else:
            #if (len(self.consequent_differs.squeeze().tolist() )>1):
            self.existential_but_for = any(self.consequent_differs.squeeze().tolist()                )  

            

        witness_dict = dict()
        if self.witness_candidates:
            witness_keys = ["__split_" + candidate for candidate in self.witness_candidates]
            witness_dict = {key: self.trace.nodes[key]['value']  for key in witness_keys}
            
        witness_dict['observed'] = self.observed_consequent.squeeze()
        witness_dict['intervened'] = self.intervened_consequent.squeeze()
        witness_dict['consequent_differs'] = self.consequent_differs.squeeze()

        # slightly hacky as above
        self.witness_df = pd.DataFrame(witness_dict) if self.witness_candidates else witness_dict


As the definition of actual causality requires a minimality check, we implement it on top of the existential but-for test:

In [3]:
def ac_minimality_check(HPM):
    HPM_with_minimality = HPM

    ante = HPM_with_minimality.antecedents 
    ante_subsets = []
    ante_existential_but_for = []
    n = len(ante)

    for r in range(n):
        subsets = combinations(ante, r)
        ante_subsets.extend([list(subset) for subset in subsets])
    
    HPM_with_minimality.ante_subsets = ante_subsets

    for sub in  range(len(HPM_with_minimality.ante_subsets)):
        HPM_sub = HalpernPearlModifiedApproximate(
        model = HPM_with_minimality.model,
        antecedents = HPM_with_minimality.ante_subsets[sub],
        outcome = HPM_with_minimality.outcome,
        witness_candidates = HPM_with_minimality.witness_candidates,
        observations = HPM_with_minimality.observations,
        sample_size = HPM_with_minimality.sample_size,
        event_dim = 0
        )
         
        HPM_sub()
        ante_existential_but_for.append(HPM_sub.existential_but_for)

    HPM_with_minimality.ante_subsets = ante_subsets
    HPM_with_minimality.ante_existential_but_for = ante_existential_but_for

    if any(HPM_with_minimality.ante_existential_but_for):
        HPM_with_minimality.minimal = False
    else:
        HPM_with_minimality.minimal = True

    HPM_with_minimality.ac = HPM_with_minimality.minimal and HPM_with_minimality.existential_but_for 

    return(HPM_with_minimality)


## Examples

### Comments on example selection



For the sake of illustration, we reconstruct a few examples, which-with one exception (friendly fire incident)-come from Halpern's book. The selection is as follows:

- **Stone throwing:** this is a classic, simple structure in which the but-for clause fails due to over-determination, but an actual causality claim holds.

- **Forest fire:** one of the simplest structures illustrating conjunctions being actual causes, and how an event can be part of an actual cause without being an actual cause itself.

- **Doctors:** a simple example illustrating the intransitivity of actual causality.

- **Friendly fire incident:** a real-life example, to illustrate how the tools can be applied outside of a narrow selection of thought experiments.

- **Voting:** this illustrates how on this approach a voter is only an actual cause if they can make a difference, but only part of an actual cause otherwise, which motivates reflection on responsibility and blame, to which we will come back in other notebooks #dTODO add links

### Stone-throwing

Sally and Billy pick up stones and throw them at a bottle. Sally's stone gets there first, shattering the bottle. Both throws are perfectly accurate, so Billy's stone would have shattered the bottle had it not been preempted by Sally’s throw. (see *Actual Causality*, p. 3 and multiple further points at which the example is discussed in the book).

In [4]:
def stones_model():        
    prob_sally_throws = pyro.sample("prob_sally_throws", dist.Beta(1, 1))
    prob_bill_throws = pyro.sample("prob_bill_throws", dist.Beta(1, 1))
    prob_sally_hits = pyro.sample("prob_sally_hits", dist.Beta(1, 1))
    prob_bill_hits = pyro.sample("prob_bill_hits", dist.Beta(1, 1))
    prob_bottle_shatters_if_sally = pyro.sample("prob_bottle_shatters_if_sally", dist.Beta(1, 1))
    prob_bottle_shatters_if_bill = pyro.sample("prob_bottle_shatters_if_bill", dist.Beta(1, 1))


    sally_throws = pyro.sample("sally_throws", dist.Bernoulli(prob_sally_throws))
    bill_throws = pyro.sample("bill_throws", dist.Bernoulli(prob_bill_throws))

    new_shp = torch.where(sally_throws == 1,prob_sally_hits , 0.0)

    sally_hits = pyro.sample("sally_hits",dist.Bernoulli(new_shp))

    new_bhp = torch.where(
            (
                bill_throws.bool()
                & (~sally_hits.bool())
            )
            == 1,
            prob_bill_hits,
            torch.tensor(0.0),
        )


    bill_hits = pyro.sample("bill_hits", dist.Bernoulli(new_bhp))

    new_bsp = torch.where(
            bill_hits.bool() == 1,
            prob_bottle_shatters_if_bill,
            torch.where(
                sally_hits.bool() == 1,
                prob_bottle_shatters_if_sally,
                torch.tensor(0.0),
            ),
        )

    bottle_shatters = pyro.sample(
            "bottle_shatters", dist.Bernoulli(new_bsp)
        )

    return {
            "sally_throws": sally_throws,
            "bill_throws": bill_throws,
            "sally_hits": sally_hits,
            "bill_hits": bill_hits,
            "bottle_shatters": bottle_shatters,
        }

stones_model.nodes = [
            "sally_throws",
            "bill_throws",
            "sally_hits",
            "bill_hits",
            "bottle_shatters",
        ]

We now instantiate the class, specifying the observations. When we run the resulting model, we then randomly generate 100 witness sets - `witness_df` contains information on whether the intervention changes the consequent for each of these preemptions. 

In [5]:
pyro.set_rng_seed(101)
stonesHPM = HalpernPearlModifiedApproximate(
    model = stones_model,
    antecedents = ["sally_throws"],
    outcome = "bottle_shatters",
    witness_candidates = ["bill_throws", "bill_hits"],
    observations = {"prob_sally_throws": 1, 
                    "prob_bill_throws": 1,
                    "prob_sally_hits": 1,
                    "prob_bill_hits": 1,
                    "prob_bottle_shatters_if_sally": 1,
                    "prob_bottle_shatters_if_bill": 1,
                    "sally_throws": 1, "bill_throws": 1},
    sample_size = 100,
    event_dim = 0
)

stonesHPM()
stonesHPM.witness_df

,__split_bill_throws,__split_bill_hits,observed,intervened,consequent_differs
0,1,0,1.0,1.0,False
1,0,1,1.0,0.0,True
2,0,0,1.0,1.0,False
3,0,0,1.0,1.0,False
4,1,1,1.0,0.0,True
...,...,...,...,...,...
95,1,0,1.0,1.0,False
96,0,1,1.0,0.0,True
97,1,1,1.0,0.0,True
98,0,1,1.0,0.0,True


The existential causality claim (*is there a witness set such that if it is intervened to be fixed at the actual values, an intervention on the antecedent to have a different value would cause the consequent to have a different value?*) holds just in case consequent differs at list ones. This information is contained in `existential_but_for`.

In [6]:
stonesHPM.existential_but_for

True

We now can use `ac_minimality_check` to check further conditions.

In [7]:
stones_min = ac_minimality_check(stonesHPM)
print(stones_min.ante_subsets) #there is only one, empty subset
print(stones_min.ante_existential_but_for) #absolute but-for clause fails
print(stones_min.minimal) #so our antecedent is minimal
print(stones_min.ac) #and the actual causality claim holds

[[]]
[False]
True
True


In [8]:
# let's compare it to a case in which 
# the antecedent is not minimal
pyro.set_rng_seed(101)
stones_redundant_HPM = HalpernPearlModifiedApproximate(
    model = stones_model,
    antecedents = ["sally_throws", "bill_throws"],
    outcome = "bottle_shatters",
    witness_candidates = ["bill_throws", "bill_hits"],
    observations = {"prob_sally_throws": 1, 
                    "prob_bill_throws": 1,
                    "prob_sally_hits": 1,
                    "prob_bill_hits": 1,
                    "prob_bottle_shatters_if_sally": 1,
                    "prob_bottle_shatters_if_bill": 1,
                    "sally_throws": 1, "bill_throws": 1},
    sample_size = 100,
    event_dim = 0
)

stones_redundant_HPM()

stones_min_checked = ac_minimality_check(stones_redundant_HPM)
print(stones_min_checked.ante_subsets)
print(stones_min_checked.ante_existential_but_for)
print(stones_min_checked.ac)

[[], ['sally_throws'], ['bill_throws']]
[False, True, False]
False


### Forest fire

In this simplified model, a forest fire was caused by lightning or an arsonist, so we use three endogenous variables, and two exogenous variables corresponding to the two factors. In the conjunctive model,
both of the factors have to be present for the fire to start. In the disjunctive model, each of them alone is sufficient.

In [9]:
def ff_conjunctive():
    u_match_dropped = pyro.sample("u_match_dropped", dist.Bernoulli(0.5))
    u_lightning = pyro.sample("u_lightning", dist.Bernoulli(0.5))

    match_dropped = pyro.deterministic("match_dropped",
                                       u_match_dropped, event_dim=0)
    lightning = pyro.deterministic("lightning", u_lightning, event_dim=0)
    forest_fire = pyro.deterministic("forest_fire", torch.logical_and(match_dropped, lightning), event_dim=0).float()

    return {"match_dropped": match_dropped, "lightning": lightning,
            "forest_fire": forest_fire}

In [10]:
def ff_disjunctive():
    u_match_dropped = pyro.sample("u_match_dropped", dist.Bernoulli(0.5))
    u_lightning = pyro.sample("u_lightning", dist.Bernoulli(0.5))

    match_dropped = pyro.deterministic("match_dropped",
                                       u_match_dropped, event_dim=0)
    lightning = pyro.deterministic("lightning", u_lightning, event_dim=0)
    forest_fire = pyro.deterministic("forest_fire", torch.logical_or(match_dropped, lightning), event_dim=0).float()

    return {"match_dropped": match_dropped, "lightning": lightning,
            "forest_fire": forest_fire}


In [11]:
# In the conjunctive model 
# Each of the two factors is a but-for cause
 
pyro.set_rng_seed(101)
ff_conjunctiveHPM = HalpernPearlModifiedApproximate(
    model = ff_conjunctive,
    antecedents = ["match_dropped"],
    outcome = "forest_fire",
    witness_candidates = ["lightning"],
    observations = {"match_dropped": 1, "lightning": 1},
    sample_size = 4,
    event_dim = 0
)

ff_conjunctiveHPM()
ff_conjunctiveHPM_min = ac_minimality_check(ff_conjunctiveHPM)
print(ff_conjunctiveHPM_min.ac) 


True


In [12]:
# In the disjunctive model 
# there still would be fire if there was no lightning

pyro.set_rng_seed(101)
ff_disjunctiveHPM = HalpernPearlModifiedApproximate(
    model = ff_disjunctive,
    antecedents = ["match_dropped"],
    outcome = "forest_fire",
    witness_candidates = ["lightning"],
    observations = {"match_dropped": 1, "lightning": 1},
    sample_size = 4,
    event_dim = 0
)

ff_disjunctiveHPM()
ff_disjunctiveHPM.existential_but_for
# no need for further checks

False

In [13]:
# in the disjunctive model
# the actual cause is the conjunction of the two factors

pyro.set_rng_seed(101)
ff_disjunctive_jointHPM = HalpernPearlModifiedApproximate(
    model = ff_disjunctive,
    antecedents = ["match_dropped", "lightning"],
    outcome = "forest_fire",
    witness_candidates = [],
    observations = {"match_dropped": 1, "lightning": 1},
    sample_size = 4,
    event_dim = 0
)

ff_disjunctive_jointHPM()
print(ff_disjunctive_jointHPM.existential_but_for)

ff_disjunctive_jointHPM_min = ac_minimality_check(ff_disjunctive_jointHPM)
print(ff_disjunctive_jointHPM_min.ante_subsets)
print(ff_disjunctive_jointHPM_min.ante_existential_but_for)
print(ff_disjunctive_jointHPM_min.ac) 

tensor(True)
[[], ['match_dropped'], ['lightning']]
[tensor(False), tensor(False), tensor(False)]
tensor(True)


### Doctors

This example illustrates that actual causality is not, in general, transitive. One doctor is responsible for administering the medicine on Monday, and if she does, Bill recovers on Tuesday.
Another doctor is reliable and treats Bill on Tuesday if the first doctor failed to do so on Monday. If both doctors treat Bill, he is in `condition1`, dead on Wednesday. Otherwise, he is either healthy on Tuesday (`condition2`) or healthy on Wednesday (`condition3`), or did not receive any treatment and feels worse but is alive on Wednesday (`condition4`).

Now suppose Bill did receive treatment on Monday. This is an actual cause of his not receiving treatment on Tuesday, and the latter is an actual cause of his being alive on Wednesday. However, there is nothing that the first doctor could do to cause Bill to be dead on Wednesday.

In [14]:
def bc_function(mt, tt):
    condition1 = (mt == 1) & (tt == 1)
    condition2 = (mt == 1) & (tt == 0)
    condition3 = (mt == 0) & (tt == 1)
    condition4 = ~(condition1 | condition2 | condition3)

    output = torch.where(condition1, torch.tensor(3.0), torch.tensor(0.0))
    output = torch.where(condition2, torch.tensor(0.0), output)
    output = torch.where(condition3, torch.tensor(1.0), output)
    output = torch.where(condition4, torch.tensor(2.0), output)

    return output


def model_doctors():
    u_monday_treatment = pyro.sample("u_monday_treatment", dist.Bernoulli(0.5))

    monday_treatment = pyro.deterministic(
        "monday_treatment", u_monday_treatment, event_dim=0
    )

    tuesday_treatment = pyro.deterministic(
        "tuesday_treatment",
        torch.logical_not(monday_treatment).float(),
        event_dim=0,
    )

    bills_condition = pyro.deterministic(
        "bills_condition",
        bc_function(monday_treatment, tuesday_treatment),
        event_dim=0,
    )

    bill_alive = pyro.deterministic(
        "bill_alive", bills_condition.not_equal(3.0).float(), event_dim=0
    )

    return {
        "monday_treatment": monday_treatment,
        "tuesday_treatment": tuesday_treatment,
        "bills_condition": bills_condition,
        "bill_alive": bill_alive,
    }

In [15]:
doctors1_HPM = HalpernPearlModifiedApproximate(
    model = model_doctors,
    antecedents = ["monday_treatment"],
    outcome = "tuesday_treatment",
    witness_candidates = [],
    observations = {"u_monday_treatment": 1},
    sample_size = 4,
    event_dim = 0
)

doctors1_HPM()

doctors2_HPM = HalpernPearlModifiedApproximate(
    model = model_doctors,
    antecedents = ["tuesday_treatment"],
    outcome = "bill_alive",
    witness_candidates = [],
    observations = {"u_monday_treatment": 1},
    sample_size = 4,
    event_dim = 0
)

doctors2_HPM()

doctors3_HPM = HalpernPearlModifiedApproximate(
    model = model_doctors,
    antecedents = ["monday_treatment"],
    outcome = "bill_alive",
    witness_candidates = [],
    observations = {"u_monday_treatment": 1},
    sample_size = 4,
    event_dim = 0
)

doctors3_HPM()


doctors1_HPM_min = ac_minimality_check(doctors1_HPM)
doctors2_HPM_min = ac_minimality_check(doctors2_HPM)
doctors3_HPM_min = ac_minimality_check(doctors3_HPM)


print(
"step 1:", doctors1_HPM.ac,
"step 2:",  doctors2_HPM.ac,
"step 3:", doctors3_HPM.ac
)

print(doctors3_HPM_min.existential_but_for)

step 1: tensor(True) step 2: tensor(True) step 3: tensor(False)
tensor(False)


### Friendly fire



This comes from a causal model developed in a real-life incident investigation, as discussed in the [Incident Reporting using SERAS® Reporter and SERAS® Analyst](http://www.causalis.com/90-publications/IncidentReportingUsingSERAS.pdf) paper.

a U.S. Special Forces air controller changing the battery on a Global Positioning System device he was using to target a Taliban outpost north of Kandahar. Three special forces soldiers were killed and 20 were injured when a 2,000-pound, satellite-guided bomb landed, not on the Taliban outpost, but on a battalion command post occupied by American forces and a group of Afghan allies, including Hamid Karzai, now the interim prime minister. The Air Force combat controller was using a Precision Lightweight GPS Receiver to calculate the Taliban's coordinates for the attack. The controller did not realize that after he changed the device's battery, the machine was programmed to automatically come back on displaying coordinates for its own location, the official said.

Minutes before the B-52 strike, the controller had used the GPS receiver to
calculate the latitude and longitude of the Taliban position in minutes and seconds for an airstrike by a Navy F/A-18. Then, with the B-52 approaching the target, the air controller did a second calculation in “degree decimals” required by the bomber crew.  The controller had performed the calculation and recorded the position, when the receiver battery died. Without realizing the machine was programmed to come back on showing the coordinates of its
own location, the controller mistakenly called in the American position to the B-52.

Factors included in the model:

1. The air controller changed the battery on the PLGR
2. Three special forces soldiers were killed and 20 were injured
3. B-52 fired a JDAM bomb at the Allied position
4. The air controller was using the PLGR to calculate the Taliban's coordinates
5. The controller did not realize that the PLGR was programmed to automatically come back on displaying coordinates for its own location
6. The controller had used the PLGR to calculate the latitude and longitude of the Taliban position in minutes and seconds for an airstrike by a Navy F/A-18
7. The air controller did a second calculation in “degree decimals” required by the bomber crew
8. The controller had performed the calculation and recorded the position
9. The controller mistakenly called in the American position to the B-52
10. The B-52 fired a JDAM bomb at the Allied position
11. The U.S. Air Force and Army had a training problem
12. The PLRG resumed displaying the coordinates of its own location after the battery was changed
13. The battery died at the crucial time
14. The controller thought he was calling in the Taliban position

The DAG used in the model is as follows:
![Friendly Fire DAG](figures/friendly_fire_dag.png)

In [16]:

def model_friendly_fire():
    u_f4_PLGR_now = pyro.sample("u_f4_PLGR_now", dist.Bernoulli(0.5))
    u_f11_training = pyro.sample("u_f11_training", dist.Bernoulli(0.5))

    f4_PLGR_now = pyro.deterministic("f4_PLGR_now", u_f4_PLGR_now, event_dim=0)
    f11_training = pyro.deterministic(
        "f11_training", u_f11_training, event_dim=0
    )

    f6_PLGR_before = pyro.deterministic(
        "f6_PLGR_before", f4_PLGR_now, event_dim=0
    )
    f7_second_calculation = pyro.deterministic(
        "f7_second_calculation", f4_PLGR_now, event_dim=0
    )
    f13_battery_died = pyro.deterministic(
        "f13_battery_died",
        f6_PLGR_before.bool() & f7_second_calculation.bool(),
        event_dim=0,
    )

    f1_battery_change = pyro.deterministic(
        "f1_battery_change", f13_battery_died, event_dim=0
    )

    f12_PLGR_after = pyro.deterministic(
        "f12_PLGR_after", f1_battery_change, event_dim=0
    )

    f5_unaware = pyro.deterministic("f5_unaware", f11_training, event_dim=0)

    f14_wrong_position = pyro.deterministic(
        "f14_wrong_position", f5_unaware, event_dim=0
    )

    f9_mistake_call = pyro.deterministic(
        "f9_mistake_call",
            f12_PLGR_after.bool() & 
            f14_wrong_position.bool(),
        event_dim=0,
    )

    f3_fired = pyro.deterministic("f3_fired", f9_mistake_call, event_dim=0)

    f10_landed = pyro.deterministic(
        "f10_landed", f3_fired.bool() &  f9_mistake_call.bool(), event_dim=0
    )

    f2_killed = pyro.deterministic("f2_killed", f10_landed, event_dim=0)

    return {
        "f1_battery_change": f1_battery_change,
        "f2_killed": f2_killed,
        "f3_fired": f3_fired,
        "f4_PLGR_now": f4_PLGR_now,
        "f5_unaware": f5_unaware,
        "f6_PLGR_before": f6_PLGR_before,
        "f7_second_calculation": f7_second_calculation,
        "f9_mistake_call": f9_mistake_call,
        "f10_landed": f10_landed,
        "f11_training": f11_training,
        "f12_PLGR_after": f12_PLGR_after,
        "f13_battery_died": f13_battery_died,
        "f14_wrong_position": f14_wrong_position,
    }



In [17]:
# while a conjunction of these two nodes satisfies the existential but-for...

friendly_fire_HPM = HalpernPearlModifiedApproximate(
    model = model_friendly_fire,
    antecedents = ["f6_PLGR_before", "f7_second_calculation"],
    outcome = "f2_killed",
    witness_candidates = ["f4_PLGR_now","f5_unaware",
    "f11_training",
    "f14_wrong_position"],
    observations = {"u_f4_PLGR_now": 1.0, "u_f11_training": 1.0},
    sample_size = 20,
    event_dim = 0
)

friendly_fire_HPM()
print(friendly_fire_HPM.existential_but_for)

# ... it is not minimal as so does any of the two factors alone
friendly_fire_HPM_min = ac_minimality_check(friendly_fire_HPM)

print(friendly_fire_HPM_min.ante_subsets)
print(friendly_fire_HPM_min.ante_existential_but_for)
print(friendly_fire_HPM_min.minimal)
print(friendly_fire_HPM_min.ac)


True
[[], ['f6_PLGR_before'], ['f7_second_calculation']]
[False, True, True]
False
False


### Voting


The main reason why the voting models are interesting in this context is that we are interested in the role of particular voters in the coming about of the result. The intuition-and we will pursue it in the responsibility notebook-is that a voter might play are role or be blamed for not voting even if her vote is not decisive. For now, we just notice that the notion of actual causality at play is not enough to capture these intuitions. Say you give one vote in a binary majority vote, `vote0`, you vote "for", and there are six other voters.  

In [18]:
def voting_model():
    u_vote0 = pyro.sample("u_vote0", dist.Bernoulli(0.6))
    u_vote1 = pyro.sample("u_vote1", dist.Bernoulli(0.6))
    u_vote2 = pyro.sample("u_vote2", dist.Bernoulli(0.6))
    u_vote3 = pyro.sample("u_vote3", dist.Bernoulli(0.6))
    u_vote4 = pyro.sample("u_vote4", dist.Bernoulli(0.6))
    u_vote5 = pyro.sample("u_vote5", dist.Bernoulli(0.6))

    vote0 = pyro.deterministic("vote0", u_vote0, event_dim=0)
    vote1 = pyro.deterministic("vote1", u_vote1, event_dim=0)
    vote2 = pyro.deterministic("vote2", u_vote2, event_dim=0)
    vote3 = pyro.deterministic("vote3", u_vote3, event_dim=0)
    vote4 = pyro.deterministic("vote4", u_vote4, event_dim=0)
    vote5 = pyro.deterministic("vote5", u_vote5, event_dim=0)
    return {"outcome": vote0 + vote1 + vote2 + vote3 + vote4 + vote5 > 3}


In [19]:
# if you're one of four voters who voted for, you are an actual cause
# of the outcome

voting4HPM = HalpernPearlModifiedApproximate(
    model = voting_model,
    antecedents = ["vote0"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(1,6)],
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
                        u_vote3=1., u_vote4=0., u_vote5=0),
    sample_size = 1000)

voting4HPM()

voting4HPM.existential_but_for

True

In [20]:
# if you're one of five voters who voted for, you are not an actual cause
# of the outcome

voting5HPM = HalpernPearlModifiedApproximate(
    model = voting_model,
    antecedents = ["vote0"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(1,6)],
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
                        u_vote3=1., u_vote4=1., u_vote5=0),
    sample_size = 1000)

voting5HPM()

voting5HPM.existential_but_for

False

In [21]:
# still, you are part of an actual cause 

voting_groupHPM = HalpernPearlModifiedApproximate(
    model = voting_model,
    antecedents = ["vote0", "vote1"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(2,6)],
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
                        u_vote3=1., u_vote4=1., u_vote5=0),
    sample_size = 1000)

voting_groupHPM()

voting_groupHPM.existential_but_for


True

## References

(Halpern 2016) Halpern, Josepy Y., "Actual Causality", MIT Press, Cambridge, 2016